In [ ]:
import findspark
findspark.init('/opt/spark')

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder\
    .enableHiveSupport()\
    .getOrCreate()

In [ ]:
spark.sql("DROP TABLE IF EXISTS src")
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")
spark.sql("LOAD DATA LOCAL INPATH '/opt/spark/examples/src/main/resources/kv1.txt' INTO TABLE src")

In [ ]:
spark.sql("SELECT * FROM src").show()

In [ ]:
spark.sql("SELECT COUNT(*) FROM src").show()

In [ ]:
sqlDF = spark.sql("SELECT key, value FROM src WHERE key < 10 ORDER BY key")

In [ ]:
stringsDS = sqlDF.rdd.map(lambda row: "Key: %d, Value: %s" % (row.key, row.value))
for record in stringsDS.collect():
    print(record)

In [ ]:
from pyspark.sql import Row

Record = Row("key", "value")
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])
recordsDF.createOrReplaceTempView("records")

In [ ]:
spark.sql("SELECT * FROM records r JOIN src s ON r.key = s.key").show()

In [ ]:
spark.stop()